In [2]:
import numpy as np
import matplotlib.pyplot as plt


from numba import jit

In [3]:
def initial_system_config(N):
    # Define a configuração inicial do sistema
    # S rcebe valores aleatórios -1 ou 1
    S = np.random.randint(0,2,N)
    S = 2 * S - 1
    return S

In [4]:
def expos(beta):
    ex = np.zeros(5,dtype=np.float32)
    ex[0]=np.exp(8.0*beta)
    ex[1]=np.exp(4.0*beta)
    ex[2]=1.0
    ex[3]=np.exp(-4.0*beta)
    ex[4]=np.exp(-8.0*beta)
    return ex

In [5]:
@jit(nopython=True)
def vizinhos(N):
    # Define a tabela de vizinhos
    L=int(np.sqrt(N))
    viz = np.zeros((N,4),dtype=np.int16)
    for k in range(N):
        viz[k,0]=k+1
        if (k+1) % L == 0:
            viz[k,0] = k+1-L
        viz[k,1] = k+L
        if k > (N-L-1):
            viz[k,1] = k+L-N
        viz[k,2] = k-1
        if (k % L == 0):
            viz[k,2] = k+L-1
        viz[k,3] = k-L
        if k < L:
            viz[k,3] = k+N-L
    return viz

In [6]:
def energy(N, viz, S):
    E = 0.0
    for i in range(N):
        h = S(viz[i,0]) + S(viz[i,1])
        E += -S(i)*h

In [7]:
def metroplis_ising(N, T):    
# Passos gerais do algoritmo
    beta = 1/T
    ex = expos(beta)
    S = initial_system_config(N)
    viz = vizinhos(N)
    r = np.random.uniform(0,1)
    for _ in range(1000):
        spin = np.random.randint(0, N-1)
        h = S[viz[spin,0]] + S[viz[spin,1]] + S[viz[spin,2]] + S[viz[spin,3]] # soma dos vizinhos
        de = int(S[spin] * h * 0.5 + 2)
        P = ex[de]
        if r <= P:
            S[spin] = -S[spin]
    return S
        

# Temperatura está em unidades de j/kB, onde kb é a constante de Boltzmann

0.5484627261190148